# Activity Log Routes

> routes for interacting with the activity log


In [50]:
# | default_exp routes.activity_log

In [51]:
# | exporti

import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

In [52]:
# | export
async def get_activity_log_object_types(auth: dmda.DomoAuth, parent_class :str=None, function_name :str=None, debug_num_stacks_to_drop=1) -> rgd.ResponseGetData:
    """retrieves a list of valid objectTypes that can be used to search the activity_log API"""

    url = f"https://{auth.domo+_instance}.domo.com/api/audit/v1/user-audits/objectTypes"

    return await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )
    if res.status != 200:
        res.is_sucess = False
        return res.response


#### sample implementation of get_activity_log_object_types


In [53]:
import os
import pandas as pd

try:
    token_auth = dmda.DomoTokenAuth(
        domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
        domo_instance="domo-community",
    )

    al_objecttype_res = await get_activity_log_object_types(auth=token_auth)

    print(pd.DataFrame(al_objecttype_res.response))

    # use to update `ActivityLog_ObjectType` enum
    # [ f"{objecttype.get('translation').upper().replace(' ', '_')} = '{objecttype.get('type')}'" for objecttype in  al_objecttype_res.response]

except Exception as e:
    print(e)

                      type             translation
0             ACCESS_TOKEN            Access token
1                  ACCOUNT                 Account
2             ACTIVITY_LOG            Activity log
3                    ALERT                   Alert
4                      APP                     App
..                     ...                     ...
76           Workbench_JOB           Workbench job
77      Workbench_SCHEDULE      Workbench schedule
78       WORKFLOW_INSTANCE       Workflow Instance
79          WORKFLOW_MODEL          Workflow Model
80  WORKFLOW_MODEL_VERSION  Workflow Model Version

[81 rows x 2 columns]


### Error Handling

In [54]:
# | export
import domolibrary.client.DomoError as de

In [55]:
class Search_Activity_Log_Error(de.DomoError):
    def __init__(self,
        parent_class :str=None,
        function_name :str=None):
        super().__init__(
            parent_class=parent_class,
            function_name=function_name)

### more routes

In [56]:
# | export
@gd.route_function
async def search_activity_log(
    auth: dmda.DomoAuth,
    start_time: int,  # epoch time in milliseconds
    end_time: int,  # epoch time in milliseconds
    maximum: int,
    object_type: str = None,
    debug_api: bool = False,
    debug_loop: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop: int = 1,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    """loops over activity log api to retrieve audit logs"""
    
    if not session: 
            session = httpx.AsyncClient()
            is_close_session = True

    url = f"https://{auth.domo_instance}.domo.com/api/audit/v1/user-audits"

    if object_type and object_type != 'ACTIVITY_LOG':
            url = f"{url}/objectTypes/{object_type}"

    fixed_params = {"end": end_time, "start": start_time}

    offset_params = {
            "offset": "offset",
            "limit": "limit",
        }

    def arr_fn(res) -> list[dict]: 
            return res.response

    res = await gd.looper(   
            auth=auth,
            method="GET",
            url=url,
            arr_fn=arr_fn,
            fixed_params=fixed_params,
            offset_params=offset_params,
            session=session,
            maximum=maximum,
            debug_loop=debug_loop,
            debug_api=debug_api,
    )
    if res.status != 200:
            res.is_sucess = False
            raise Search_Activity_Log_Error( 
                   domo_instance=dmda.DomoAuth, 
                   status=res.status, 
                   message=res.response,
                   entity_id="entity_id"
        )

    if is_close_session: 
            await session.aclose()

    return res

In [57]:
import os
import pandas as pd
import datetime as dt

import domolibrary.utils.convert as convert

end_datetime = dt.datetime.today()
start_datetime = end_datetime - dt.timedelta(days=100)

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-community",
)

page_res = await search_activity_log(
    object_type="PAGE",
    start_time=convert.convert_datetime_to_epoch_millisecond(start_datetime),
    end_time=convert.convert_datetime_to_epoch_millisecond(end_datetime),
    auth=token_auth,
    maximum=16,
    debug_loop=False,
    debug_api=False
)

pd.DataFrame(page_res.response[0:5])

,objectId,objectName
0,-100000,Overview
1,-100001,Alerts
2,-100002,Shared
3,-100003,Favorites
4,-100012,CEO


#### sample implementation of search_activity_log


In [58]:
import os
import pandas as pd
import datetime as dt

import domolibrary.utils.convert as convert

end_datetime = dt.datetime.today()
start_datetime = end_datetime - dt.timedelta(days=100)

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-community",
)

activity_log_res = await search_activity_log(
    object_type="ACTIVITY_LOG",
    start_time=convert.convert_datetime_to_epoch_millisecond(start_datetime),
    end_time=convert.convert_datetime_to_epoch_millisecond(end_datetime),
    auth=token_auth,
    maximum=10,
    debug_loop=False,
    debug_api=False
)

pd.DataFrame(activity_log_res.response[0:5])

,userName,userId,userType,actorName,actorId,actionType,objectName,objectId,objectType,additionalComment,time,eventText,clientId,authenticationMethod
0,Bryan Van Kampen,587894148,USER,,0,VIEWED,Workbench Monitoring,1416746303,PAGE,Bryan Van Kampen viewed page Workbench Monitor...,1698098686062,Viewed page,None,DEV_TOKEN
1,Bryan Van Kampen,587894148,USER,,0,VIEWED,Welcome,30507758,PAGE,Bryan Van Kampen viewed page Welcome.,1698098685592,Viewed page,None,DEV_TOKEN
2,Bryan Van Kampen,587894148,USER,,0,VIEWED,Virtual User Group,138471350,PAGE,Bryan Van Kampen viewed page Virtual User Group.,1698098683705,Viewed page,None,DEV_TOKEN
3,Bryan Van Kampen,587894148,USER,,0,VIEWED,Time Zone Handling,950006533,PAGE,Bryan Van Kampen viewed page Time Zone Handling.,1698098682200,Viewed page,None,DEV_TOKEN
4,Bryan Van Kampen,587894148,USER,,0,VIEWED,Text Analytics,225044251,PAGE,Bryan Van Kampen viewed page Text Analytics.,1698098681870,Viewed page,None,DEV_TOKEN


In [59]:
# | hide
import nbdev

nbdev.nbdev_export()
!nbqa black activity_log.ipynb